In [1]:
"""
Script de visualisation de base pour le dataset de tomographie de qubit.

- Génère un dataset de tomographie (fonction generate_qubit_tomography_dataset_base)
- Génère un dataset de classification pure/mixte (build_purity_classification_dataset)
- Produit plusieurs images prêtes à être incluses dans le rapport LaTeX :
    * bloch_ideal.png
    * bloch_real.png
    * bloch_purity.png
    * hist_bloch_radius.png
"""

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 (nécessaire pour la projection 3d)
import os

# Adapter le chemin si besoin
from saint_dtSet import (
    generate_qubit_tomography_dataset_base,
    build_purity_classification_dataset,
)


# ============================
# 1. Fonctions utilitaires
# ============================

def ensure_output_dir(path: str = "figures"):
    """Crée le dossier de sortie s'il n'existe pas."""
    if not os.path.exists(path):
        os.makedirs(path)
    return path


def plot_bloch_sphere(ax, radius: float = 1.0, alpha: float = 0.1):
    """
    Dessine une sphère de Bloch (surface) dans un axe 3D existant.
    """
    # Grille sphérique
    u = np.linspace(0, 2 * np.pi, 60)
    v = np.linspace(0, np.pi, 30)
    x = radius * np.outer(np.cos(u), np.sin(v))
    y = radius * np.outer(np.sin(u), np.sin(v))
    z = radius * np.outer(np.ones_like(u), np.cos(v))

    ax.plot_surface(
        x, y, z,
        rstride=1, cstride=1,
        linewidth=0, antialiased=True,
        alpha=alpha
    )

    # Axes
    ax.set_xlabel(r"$X$")
    ax.set_ylabel(r"$Y$")
    ax.set_zlabel(r"$Z$")
    ax.set_xlim([-radius, radius])
    ax.set_ylim([-radius, radius])
    ax.set_zlim([-radius, radius])
    ax.set_box_aspect([1, 1, 1])


def scatter_on_bloch(ax, x, y, z, c=None, s: int = 10, cmap="viridis", label=None):
    """
    Ajoute un nuage de points sur un axe 3D.
    """
    sc = ax.scatter(x, y, z, c=c, s=s, cmap=cmap, depthshade=True, label=label)
    if c is not None:
        # On ajoute une barre de couleur si on code la couleur par une grandeur continue
        cb = plt.colorbar(sc, ax=ax, shrink=0.7)
        cb.ax.set_ylabel("Code couleur")
    if label is not None:
        ax.legend(loc="upper left")


# ============================
# 2. Figures principales
# ============================

def fig_bloch_ideal(df, out_dir="figures"):
    """
    Bloch sphere des états idéaux (X_ideal, Y_ideal, Z_ideal).
    """
    fig = plt.figure(figsize=(6, 5))
    ax = fig.add_subplot(111, projection="3d")

    plot_bloch_sphere(ax, radius=1.0, alpha=0.12)

    ax.scatter(
        df["X_ideal"],
        df["Y_ideal"],
        df["Z_ideal"],
        s=5,
        alpha=0.7
    )
    ax.set_title("États idéaux sur la sphère de Bloch")

    fig.tight_layout()
    path = os.path.join(out_dir, "bloch_ideal.png")
    fig.savefig(path, dpi=300)
    plt.close(fig)
    print(f"Figure sauvegardée : {path}")


def fig_bloch_real(df, out_dir="figures"):
    """
    Bloch sphere des états RÉELS (X_real, Y_real, Z_real), potentiellement mixtes.
    Couleur = rayon de Bloch réel.
    """
    # Rayon de Bloch réel
    r = np.sqrt(df["X_real"]**2 + df["Y_real"]**2 + df["Z_real"]**2)

    fig = plt.figure(figsize=(6, 5))
    ax = fig.add_subplot(111, projection="3d")

    plot_bloch_sphere(ax, radius=1.0, alpha=0.12)

    scatter_on_bloch(
        ax,
        df["X_real"],
        df["Y_real"],
        df["Z_real"],
        c=r,
        s=8,
        cmap="viridis",
    )
    ax.set_title("États réels (avec décohérence) sur la sphère de Bloch")

    fig.tight_layout()
    path = os.path.join(out_dir, "bloch_real.png")
    fig.savefig(path, dpi=300)
    plt.close(fig)
    print(f"Figure sauvegardée : {path}")


def fig_bloch_purity(df_clf, out_dir="figures"):
    """
    Bloch sphere pour le dataset de classification pure/mixte.
    - Points rouges : états purs
    - Points bleus : états mixtes
    """
    fig = plt.figure(figsize=(6, 5))
    ax = fig.add_subplot(111, projection="3d")

    plot_bloch_sphere(ax, radius=1.0, alpha=0.12)

    # États purs
    pure = df_clf["label_purity"] == 1
    mixte = df_clf["label_purity"] == 0

    ax.scatter(
        df_clf.loc[pure, "X_real"],
        df_clf.loc[pure, "Y_real"],
        df_clf.loc[pure, "Z_real"],
        s=10,
        alpha=0.9,
        color="tab:red",
        label="États purs"
    )

    ax.scatter(
        df_clf.loc[mixte, "X_real"],
        df_clf.loc[mixte, "Y_real"],
        df_clf.loc[mixte, "Z_real"],
        s=10,
        alpha=0.7,
        color="tab:blue",
        label="États mixtes"
    )

    ax.legend(loc="upper left")
    ax.set_title("Classification pure / mixte sur la sphère de Bloch")

    fig.tight_layout()
    path = os.path.join(out_dir, "bloch_purity.png")
    fig.savefig(path, dpi=300)
    plt.close(fig)
    print(f"Figure sauvegardée : {path}")


def fig_hist_bloch_radius(df, out_dir="figures"):
    """
    Histogramme du rayon de Bloch réel, pour illustrer la distribution de pureté.
    """
    r = np.sqrt(df["X_real"]**2 + df["Y_real"]**2 + df["Z_real"]**2)

    fig, ax = plt.subplots(figsize=(6, 4))
    ax.hist(r, bins=40, density=False, alpha=0.8)
    ax.set_xlabel(r"Rayon de Bloch réel $r = \sqrt{X^2 + Y^2 + Z^2}$")
    ax.set_ylabel("Nombre d'états")
    ax.set_title("Distribution du rayon de Bloch réel")

    fig.tight_layout()
    path = os.path.join(out_dir, "hist_bloch_radius.png")
    fig.savefig(path, dpi=300)
    plt.close(fig)
    print(f"Figure sauvegardée : {path}")


# ============================
# 3. Main
# ============================

def main():
    out_dir = ensure_output_dir("figures")

    # ---------------------------
    # Dataset de base pour tomographie
    # ---------------------------
    df_base = generate_qubit_tomography_dataset_base(
        n_states=5000,
        n_shots=1000,
        mode="finite_shots",         # bruit statistique + décohérence
        include_ideal=True,
        include_csv=False,
        include_decoherence=True,
        decoherence_level=0.6,
        random_state=42,
    )

    # Figures associées à la tomographie de base
    fig_bloch_ideal(df_base, out_dir=out_dir)
    fig_bloch_real(df_base, out_dir=out_dir)
    fig_hist_bloch_radius(df_base, out_dir=out_dir)

    # ---------------------------
    # Dataset de classification pure / mixte
    # ---------------------------
    df_clf, X_clf, y_clf = build_purity_classification_dataset(
        n_states_total=5000,
        mixed_proportion=0.5,   # 50% mixtes, 50% purs (ou approchant)
    )

    fig_bloch_purity(df_clf, out_dir=out_dir)


if __name__ == "__main__":
    main()


Figure sauvegardée : figures\bloch_ideal.png
Figure sauvegardée : figures\bloch_real.png
Figure sauvegardée : figures\hist_bloch_radius.png
Figure sauvegardée : figures\bloch_purity.png
